In [1]:
import pandas as pd
import csv
from datetime import datetime, timedelta

In [2]:
'''
Instructions:
1. Put input CSVs inside the directory specified in the input_folder variable
2. Change the date_subject to the date today. Follow the format YYYYMMDD
3. Run the notebook
4. Get the output from the output folder
5. Upload the files to https://drive.google.com/drive/folders/1QelvFeULpkytJxkWF8XnBOpK7sV6Pk80

'''

date_subject = '20201231'
date_today =  '20210114'
output_date = (datetime.now() - (datetime.strptime(date_today, '%Y%m%d')- datetime.strptime(date_subject, '%Y%m%d')))
aa_date = (datetime.now() - (datetime.strptime(date_today, '%Y%m%d')- datetime.strptime(date_subject, '%Y%m%d')) - timedelta(2))

input_folder = 'Sources/'
lookup_folder = 'Lookup Files/'
output_folder = 'For Validation/'

print(date_subject)
print(date_today)
print(type(output_date) )
print(aa_date)



20210101
20210114
<class 'datetime.datetime'>
2020-12-30 17:48:28.205200


In [3]:
def movecol(df, cols_to_move=[], ref_col='', place='After'):
    
    cols = df.columns.tolist()
    if place == 'After':
        seg1 = cols[:list(cols).index(ref_col) + 1]
        seg2 = cols_to_move
    if place == 'Before':
        seg1 = cols[:list(cols).index(ref_col)]
        seg2 = cols_to_move + [ref_col]
    
    seg1 = [i for i in seg1 if i not in seg2]
    seg3 = [i for i in cols if i not in seg1 + seg2]
    
    return(df[seg1 + seg2 + seg3])

In [4]:
# Lookup files for activity window-date lookup

d7 = pd.read_csv(lookup_folder+"7day.csv")
d30 = pd.read_csv(lookup_folder+"30day.csv")
d90 = pd.read_csv(lookup_folder+"90day.csv")
d365 = pd.read_csv(lookup_folder+"365day.csv")

from functools import reduce
window_date_lookup = reduce(lambda left,right: pd.merge(left,right,on='date'), [d7, d30, d90, d365])
window_date_lookup.columns = ['date', '7d', '30d', '90d', '365d']
window_date_lookup['date'] = pd.to_datetime(window_date_lookup['date']) 
window_date_lookup.set_index('date', inplace=True)
window_date_lookup

,7d,30d,90d,365d
date,,,,
2019-05-27,1,898,1795,2692
2019-05-28,2,899,1796,2693
2019-05-29,3,900,1797,2694
2019-05-30,4,901,1798,2695
2019-05-31,5,902,1799,2696
...,...,...,...,...
2019-04-03,8194,8201,8208,8215
2019-04-04,8195,8202,8209,8216
2019-04-05,8196,8203,8210,8217


In [5]:
country_lookup = pd.read_csv(lookup_folder+"countries_mapped_iso3_metadata.csv")
country_lookup.set_index('adobe_analytics_country', inplace=True)
country_lookup = country_lookup[['iso3']]
country_lookup

,iso3
adobe_analytics_country,
andorra,AND
austria,AUT
azerbaijan,AZE
belize,BLZ
cayman islands,CYM
...,...
russian federation,RUS
st. helena,SHN
syrian arab republic,SYR


In [6]:
# Step 6

# Change input and output directories
input_filename_6 = input_folder+'GDS_WIG3_-_Mobilizing_Digital_Missionaries - '+ str(date_subject)+'.csv'
output_filename_6 = output_folder+output_date.strftime('%Y%m%d') + '_step6_29.csv'  # default to today

input = open(input_filename_6, 'rb')

pre_csv = list(input.readlines())
rows_to_remove = [r - 1 for r in [13,11,10,9,8,7,6,5,4,3,2,1]]

for r in rows_to_remove:
    pre_csv.pop(r)


pre_csv=[x.rstrip().decode('utf-8').split(",") for x in pre_csv]
    
df = pd.DataFrame(pre_csv[1:pre_csv.index([''])],columns=pre_csv[0])


df.columns = ['country', '365d', '90d', '30d', '7d']

df.at[0, 'country'] = 'Global'
df['dateValue'] = aa_date.strftime('%Y-%m-%d')  # default to yesterday, if report is generated today
df['product'] = 'godtools'

df = movecol(df, 
             cols_to_move=['product','country', '7d', '30d', '90d', '365d'], 
             ref_col='dateValue',
             place='After')


df = df[~df.country.isin(['', None])]
df = df.dropna(subset=['country', '7d']) 


df.columns = ['date','product','country','7d','30d','90d','365d']
df['source_id'] = 29
df = df.melt(id_vars=['date','product', 'source_id', 'country'],var_name = 'window', value_name = 'value')
df['date'] =  pd.to_datetime(df['date']) 
df.country = [str(x).lower() for x in df.country]

df['isOriginal'] = 1
df['activitywindow_date_id'] = window_date_lookup.lookup(df.date, df.window)
temp=df.join(country_lookup, on='country')
temp.country = temp.iso3
temp.drop(columns=['iso3'], inplace=True)
df = temp

final_step = df.copy()

final_step['record_id'] = ''
final_step.drop(columns=['date', 'product', 'window'], inplace=True)
final_step = movecol(final_step, 
             cols_to_move=['activitywindow_date_id', 'source_id', 'country', 'value', 'isOriginal'], 
             ref_col='record_id',
             place='After')

final_step.to_csv(output_filename_6, encoding='utf-8', header=True, index=False)
final_step




FileNotFoundError: [Errno 2] No such file or directory: 'Sources/GDS_WIG3_-_Mobilizing_Digital_Missionaries - 20210101.csv'

In [ ]:
# Step 6.5

input_filename_65 = input_folder+'GDS_WIG3_-_Mobilizing_Digital_Missionaries - '+ str(date_subject)+'.csv'
output_filename_65 = output_folder+output_date.strftime('%Y%m%d') + '_step6_30.csv'  # default to today

input = open(input_filename_65, 'rb')

pre_csv = list(input.readlines())
pre_csv=[x.rstrip().decode('utf-8').split(",") for x in pre_csv]
pre_csv = pre_csv[pre_csv.index(['# MissionHub Digital Missionaries']):]
rows_to_remove = [r - 1 for r in [12,11,10,9,8,7,6,5,4,3,2,1]]

for r in rows_to_remove:
    pre_csv.pop(r)


    
df = pd.DataFrame(pre_csv[0:],columns=pre_csv[0])
df.columns = ['country', '365d', '90d', '30d', '7d']
df.at[0, 'country'] = 'Global'
df['dateValue'] = aa_date.strftime('%Y-%m-%d')  # default to yesterday, if report is generated today
df['product'] = 'missionhub'
df

df = movecol(df, 
             cols_to_move=['product', 'country', '7d', '30d', '90d', '365d'], 
             ref_col='dateValue',
             place='After')

df = df[~df.country.isin(['', 'Countries', None])]
df = df.dropna(subset=['country', '7d']) 

df.to_csv(output_filename_65, encoding='utf-8', header=False, index=False)

df.columns = ['date','product','country','7d','30d','90d','365d']
df['source_id'] = 30
df = df.melt(id_vars=['date','product', 'source_id', 'country'],var_name = 'window', value_name = 'value')
df['date'] =  pd.to_datetime(df['date']) 
df.country = [str(x).lower() for x in df.country]

df['isOriginal'] = 1
df['activitywindow_date_id'] = window_date_lookup.lookup(df.date, df.window)
temp=df.join(country_lookup, on='country')
temp.country = temp.iso3
temp.drop(columns=['iso3'], inplace=True)
df = temp

final_step = df.copy()

final_step['record_id'] = ''
final_step.drop(columns=['date', 'product', 'window'], inplace=True)
final_step = movecol(final_step, 
             cols_to_move=['activitywindow_date_id', 'source_id', 'country', 'value', 'isOriginal'], 
             ref_col='record_id',
             place='After')

final_step.to_csv(output_filename_65, encoding='utf-8', header=True, index=False)
final_step

In [ ]:
# Step 8

input_filename_8 = input_folder+'Signups by country - '+ str(date_subject)+'.csv'
output_filename_8 = output_folder+output_date.strftime('%Y%m%d') + '_step8.csv'  # default to today

input = open(input_filename_8, 'rb')

pre_csv = list(input.readlines())
pre_csv=[x.rstrip().decode('utf-8').split(",") for x in pre_csv]
rows_to_remove = [r - 1 for r in [13,12,11,10,9,8,7,6,5,4,3,2,1]]

for r in rows_to_remove:
    pre_csv.pop(r)
    

df = pd.DataFrame(pre_csv[0:],columns=pre_csv[0])

df.columns = ['country', '7d', '30d', '90d', '365d']
df['product'] = 'es-email-web-signups'
df['threshold'] = 'curious'
df['type'] = 'email'
df['subtype'] = ''
df['dateValue'] = aa_date.strftime('%Y-%m-%d')  # default to 2 days ago

df = movecol(df, 
             cols_to_move=[ 'product', '7d', '30d', '90d', '365d', 'threshold', 'country', 'type', 'subtype'],
             ref_col='dateValue',
             place='After')

df = df[~df.country.isin(['', None])]
df.at[0, 'country'] = 'Global'


df.columns = ['date','product','7d','30d','90d','365d','threshold', 'country', 'email', 'nan']
df['source_id'] = 76
df['threshold'] = 1 # Curious
df = df.melt(id_vars=['date','product', 'source_id', 'country', 'email', 'threshold', 'nan'],var_name = 'window', value_name = 'value')
df['date'] =  pd.to_datetime(df['date']) 
df.country = [str(x).lower() for x in df.country]

df['isOriginal'] = 1
df['activitywindow_date_id'] = window_date_lookup.lookup(df.date, df.window)
temp=df.join(country_lookup, on='country')
temp.country = temp.iso3
temp.drop(columns=['iso3'], inplace=True)
df = temp

final_step = df.copy()

final_step['record_id'] = ''
final_step.drop(columns=['date', 'product', 'window', 'email', 'nan'], inplace=True)
final_step = movecol(final_step, 
             cols_to_move=['activitywindow_date_id', 'source_id', 'country', 'value', 'isOriginal', 'threshold'], 
             ref_col='record_id',
             place='After')
final_step.to_csv(output_filename_8, encoding='utf-8', header=True, index=False)

final_step

In [ ]:
# Step 15

input_filename_15 = input_folder+'Exposures by Country - Cru Global Report Suite - '+ str(date_subject)+'.csv'
output_filename_15 = output_folder+output_date.strftime('%Y%m%d') + '_step15.csv'  # default to today


input = open(input_filename_15, 'rb')

pre_csv = list(input.readlines())
rows_to_remove = [r - 1 for r in [12,10,9,8,7,6,5,4,3,2,1]]

for r in rows_to_remove:
    pre_csv.pop(r)


pre_csv=[x.rstrip().decode('utf-8').split(",") for x in pre_csv]
    
df = pd.DataFrame(pre_csv[1:],columns=pre_csv[0])
df.columns = ['country', '7d', '30d', '90d', '365d']
df['dateValue'] = aa_date.strftime('%Y-%m-%d')  # default to 2 days ago, if report is generated today
df['source'] = 'adobe-analytics'

df = movecol(df, 
             cols_to_move=['source', 'country', '7d', '30d', '90d', '365d'], 
             ref_col='dateValue',
             place='After')

df = df[~df.country.isin(['', None])]
df.at[0, 'country'] = 'Global'


df.columns = ['date','product','country','7d','30d','90d','365d']
df['source_id'] = 50
df = df.melt(id_vars=['date','product', 'source_id', 'country'],var_name = 'window', value_name = 'value')
df['date'] =  pd.to_datetime(df['date']) 
df.country = [str(x).lower() for x in df.country]

df['isOriginal'] = 1
df['activitywindow_date_id'] = window_date_lookup.lookup(df.date, df.window)
temp=df.join(country_lookup, on='country')
temp.country = temp.iso3
temp.drop(columns=['iso3'], inplace=True)
df = temp

final_step = df.copy()

final_step['record_id'] = ''
final_step.drop(columns=['date', 'product', 'window'], inplace=True)
final_step = movecol(final_step, 
             cols_to_move=['activitywindow_date_id', 'source_id', 'country', 'value', 'isOriginal'], 
             ref_col='record_id',
             place='After')
final_step.to_csv(output_filename_15, encoding='utf-8', header=True, index=False)

final_step

        

In [ ]:
# Source 92

input_filename_6 = input_folder+'Decision_Tracking_for_GDS_Dashboard - '+ str(date_subject)+'.csv'
output_filename_6 = output_folder+output_date.strftime('%Y%m%d') + '_source_92.csv'  # default to today

input = open(input_filename_6, 'rb')

pre_csv = list(input.readlines())
rows_to_remove = [r - 1 for r in [12,11,10,9,8,7,6,5,4,3,2,1]]

for r in rows_to_remove:
    pre_csv.pop(r)


pre_csv=[x.rstrip().decode('utf-8').split(",") for x in pre_csv]
    
df = pd.DataFrame(pre_csv[1:pre_csv.index([''])],columns=pre_csv[0])


df.columns = ['country', '365d', '90d', '30d', '7d']

df.at[0, 'country'] = 'Global'
df['dateValue'] = aa_date.strftime('%Y-%m-%d')  # default to yesterday, if report is generated today
df['product'] = 'godtools'

df = movecol(df, 
             cols_to_move=['product','country', '7d', '30d', '90d', '365d'], 
             ref_col='dateValue',
             place='After')


df = df[~df.country.isin(['', None])]
df = df.dropna(subset=['country', '7d']) 

df.columns = ['date','product','country','7d','30d','90d','365d']
df['source_id'] = 92
df = df.melt(id_vars=['date','product', 'source_id', 'country'],var_name = 'window', value_name = 'value')
df['date'] =  pd.to_datetime(df['date']) 
df.country = [str(x).lower() for x in df.country]

df['isOriginal'] = 1
df['activitywindow_date_id'] = window_date_lookup.lookup(df.date, df.window)
temp=df.join(country_lookup, on='country')
temp.country = temp.iso3
temp.drop(columns=['iso3'], inplace=True)
df = temp

final_step = df.copy()

final_step['record_id'] = ''
final_step.drop(columns=['date', 'product', 'window'], inplace=True)
final_step = movecol(final_step, 
             cols_to_move=['activitywindow_date_id', 'source_id', 'country', 'value', 'isOriginal'], 
             ref_col='record_id',
             place='After')

final_step.to_csv(output_filename_6, encoding='utf-8', header=True, index=False)
final_step




In [ ]:
# Source 93

input_filename_6 = input_folder+'Decision_Tracking_for_GDS_Dashboard - '+ str(date_subject)+'.csv'
output_filename_6 = output_folder+output_date.strftime('%Y%m%d') + '_source_93.csv'  # default to today

input = open(input_filename_6, 'rb')

pre_csv = list(input.readlines())
pre_csv=[x.rstrip().decode('utf-8').split(",") for x in pre_csv]
pre_csv = pre_csv[pre_csv.index(['# source 93- LMI - win (non-godtools)']):pre_csv.index(['"# source 94 - Decisions for Christ (ES- pageview method) "'])]
rows_to_remove = [r - 1 for r in [4,3,2,1]]

for r in rows_to_remove:
    pre_csv.pop(r)

df = pd.DataFrame(pre_csv[1:pre_csv.index([''])],columns=pre_csv[0])

df.columns = ['country', '90d', '30d', '7d']

df.at[0, 'country'] = 'Global'
df['dateValue'] = aa_date.strftime('%Y-%m-%d')  # default to yesterday, if report is generated today
df['product'] = 'godtools'

df = movecol(df, 
             cols_to_move=['product','country', '7d', '30d', '90d'], 
             ref_col='dateValue',
             place='After')


df = df[~df.country.isin(['', None])]
df = df.dropna(subset=['country', '7d']) 

#df.to_csv(output_filename_6, encoding='utf-8', header=False, index=False)



df.columns = ['date','product','country','7d','30d','90d']
df['source_id'] = 93
df = df.melt(id_vars=['date','product', 'source_id', 'country'],var_name = 'window', value_name = 'value')
df['date'] =  pd.to_datetime(df['date']) 
df.country = [str(x).lower() for x in df.country]

df['isOriginal'] = 1
df['activitywindow_date_id'] = window_date_lookup.lookup(df.date, df.window)
temp=df.join(country_lookup, on='country')
temp.country = temp.iso3
temp.drop(columns=['iso3'], inplace=True)
df = temp

final_step = df.copy()

final_step['record_id'] = ''
final_step.drop(columns=['date', 'product', 'window'], inplace=True)
final_step = movecol(final_step, 
             cols_to_move=['activitywindow_date_id', 'source_id', 'country', 'value', 'isOriginal'], 
             ref_col='record_id',
             place='After')

final_step.to_csv(output_filename_6, encoding='utf-8', header=True, index=False)
final_step


In [ ]:
# Source 94

input_filename_6 = input_folder+'Decision_Tracking_for_GDS_Dashboard - '+ str(date_subject)+'.csv'
output_filename_6 = output_folder+output_date.strftime('%Y%m%d') + '_source_94.csv'  # default to today

input = open(input_filename_6, 'rb')

pre_csv = list(input.readlines())
pre_csv=[x.rstrip().decode('utf-8').split(",") for x in pre_csv]
pre_csv = pre_csv[pre_csv.index(['"# source 94 - Decisions for Christ (ES- pageview method) "']):pre_csv.index(['# source 95 - Decisions for Christ (godtools) app-method'
])]
rows_to_remove = [r - 1 for r in [3,2,1]]

for r in rows_to_remove:
    pre_csv.pop(r)

    
df = pd.DataFrame(pre_csv[1:pre_csv.index([''])],columns=pre_csv[0])


df.columns = ['country', '365d', '90d', '30d', '7d']

df.at[0, 'country'] = 'Global'
df['dateValue'] = aa_date.strftime('%Y-%m-%d')  # default to yesterday, if report is generated today
df['product'] = 'godtools'

df = movecol(df, 
             cols_to_move=['product','country', '7d', '30d', '90d', '365d'], 
             ref_col='dateValue',
             place='After')


df = df[~df.country.isin(['', None])]
df = df.dropna(subset=['country', '7d']) 

#df.to_csv(output_filename_6, encoding='utf-8', header=False, index=False)



df.columns = ['date','product','country','7d','30d','90d','365d']
df['source_id'] = 94
df = df.melt(id_vars=['date','product', 'source_id', 'country'],var_name = 'window', value_name = 'value')
df['date'] =  pd.to_datetime(df['date']) 
df.country = [str(x).lower() for x in df.country]

df['isOriginal'] = 1
df['activitywindow_date_id'] = window_date_lookup.lookup(df.date, df.window)
temp=df.join(country_lookup, on='country')
temp.country = temp.iso3
temp.drop(columns=['iso3'], inplace=True)
df = temp

final_step = df.copy()

final_step['record_id'] = ''
final_step.drop(columns=['date', 'product', 'window'], inplace=True)
final_step = movecol(final_step, 
             cols_to_move=['activitywindow_date_id', 'source_id', 'country', 'value', 'isOriginal'], 
             ref_col='record_id',
             place='After')

final_step.to_csv(output_filename_6, encoding='utf-8', header=True, index=False)
final_step


In [ ]:
# Source 95

input_filename_6 = input_folder+'Decision_Tracking_for_GDS_Dashboard - '+ str(date_subject)+'.csv'
output_filename_6 = output_folder+output_date.strftime('%Y%m%d') + '_source_95.csv'  # default to today

input = open(input_filename_6, 'rb')

pre_csv = list(input.readlines())
pre_csv=[x.rstrip().decode('utf-8').split(",") for x in pre_csv]
pre_csv = pre_csv[pre_csv.index(['# source 95 - Decisions for Christ (godtools) app-method']):pre_csv.index(['# 30-Day Window'
])]
rows_to_remove = [r - 1 for r in [3,2,1]]

for r in rows_to_remove:
    pre_csv.pop(r)

    
df = pd.DataFrame(pre_csv[1:pre_csv.index([''])],columns=pre_csv[0])


df.columns = ['country', '365d', '90d', '30d', '7d']

df.at[0, 'country'] = 'Global'
df['dateValue'] = aa_date.strftime('%Y-%m-%d')  # default to yesterday, if report is generated today
df['product'] = 'godtools'

df = movecol(df, 
             cols_to_move=['product','country', '7d', '30d', '90d', '365d'], 
             ref_col='dateValue',
             place='After')


df = df[~df.country.isin(['', None])]
df = df.dropna(subset=['country', '7d']) 

#df.to_csv(output_filename_6, encoding='utf-8', header=False, index=False)



df.columns = ['date','product','country','7d','30d','90d','365d']
df['source_id'] = 95
df = df.melt(id_vars=['date','product', 'source_id', 'country'],var_name = 'window', value_name = 'value')
df['date'] =  pd.to_datetime(df['date']) 
df.country = [str(x).lower() for x in df.country]

df['isOriginal'] = 1
df['activitywindow_date_id'] = window_date_lookup.lookup(df.date, df.window)
temp=df.join(country_lookup, on='country')
temp.country = temp.iso3
temp.drop(columns=['iso3'], inplace=True)
df = temp

final_step = df.copy()

final_step['record_id'] = ''
final_step.drop(columns=['date', 'product', 'window'], inplace=True)
final_step = movecol(final_step, 
             cols_to_move=['activitywindow_date_id', 'source_id', 'country', 'value', 'isOriginal'], 
             ref_col='record_id',
             place='After')

final_step.to_csv(output_filename_6, encoding='utf-8', header=True, index=False)
final_step

